# Download and Load MABe Mouse Behavior Detection Dataset

This notebook downloads the [MABe Mouse Behavior Detection](https://www.kaggle.com/competitions/MABe-mouse-behavior-detection) competition dataset using the Python `kaggle` module.

## Prerequisites
1. **Kaggle Account**: Ensure you have joined the competition on Kaggle to accept the rules.
2. **API Token**: You need a `kaggle.json` file (API Token) from your account settings.
   - Place it in `~/.kaggle/kaggle.json` (Linux/Mac) or `%USERPROFILE%\.kaggle\kaggle.json` (Windows).
   - Or set `KAGGLE_USERNAME` and `KAGGLE_KEY` environment variables.

## Manual Dataset Placement

If you have already downloaded the dataset manually, please create a directory named `data` in the same location as this notebook.
Then, place the **unzipped contents** of the competition dataset directly into this `data` directory.

For example, if you downloaded `MABe-mouse-behavior-detection.zip`, after unzipping it, you would copy the folders and files (e.g., `train`, `test`, `sample_submission.csv`) directly into `./data/`.

If you proceed this way, you can skip running the download cells (Cells 3 and 4) in this notebook.

In [1]:
# Install necessary libraries
!pip install pandas

In [2]:
import os
DATA_DIR = './data'

# Inspect the downloaded files
print("Downloaded files:")
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        print(os.path.join(root, file))

Downloaded files:
./data/.DS_Store
./data/test.csv
./data/train.csv
./data/sample_submission.csv
./data/train_tracking/.DS_Store
./data/train_tracking/PleasantMeerkat/1749697095.parquet
./data/train_tracking/PleasantMeerkat/1266446081.parquet
./data/train_tracking/PleasantMeerkat/599439.parquet
./data/train_tracking/PleasantMeerkat/886803007.parquet
./data/train_tracking/PleasantMeerkat/74645358.parquet
./data/train_tracking/PleasantMeerkat/932755484.parquet
./data/train_tracking/PleasantMeerkat/1445631188.parquet
./data/train_tracking/PleasantMeerkat/1769027005.parquet
./data/train_tracking/PleasantMeerkat/1350372183.parquet
./data/train_tracking/PleasantMeerkat/2002810725.parquet
./data/train_tracking/PleasantMeerkat/843765839.parquet
./data/train_tracking/PleasantMeerkat/908567232.parquet
./data/train_tracking/PleasantMeerkat/1375833299.parquet
./data/train_tracking/PleasantMeerkat/1450482262.parquet
./data/train_tracking/PleasantMeerkat/331245147.parquet
./data/train_tracking/Pleas

In [3]:
# Example: Load a CSV file (adjust the filename based on the output above)
import pandas as pd

# Common files in competitions: 'train.csv', 'test.csv', 'sample_submission.csv'
# Update this variable with a file found in the previous step
TARGET_FILE = f'{DATA_DIR}/sample_submission.csv' 

if os.path.exists(TARGET_FILE):
    print(f"\nLoading {TARGET_FILE}...")
    try:
        df = pd.read_csv(TARGET_FILE)
        display(df.head())
        print(f"Loaded dataframe with shape: {df.shape}")
    except Exception as e:
        print(f"Could not load file: {e}")
else:
    print(f"\nFile '{TARGET_FILE}' not found. Please check the file list above and update 'TARGET_FILE'.")


Loading ./data/sample_submission.csv...


,row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,0,438887472,mouse1,mouse2,sniff,0,1


Loaded dataframe with shape: (1, 7)
